In [56]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.svm import SVR
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings('ignore')


In [57]:
# Load data and finalize dataset variables
df = pd.read_csv('GlobalYearly.csv')

# Drop index column from csv
df = df.drop('Unnamed: 0',axis=1)

df.tail(5)

,Year,Population,Gas consumption,Coal consumption,Oil consumption,FossilFuelGrowth,CoalGrowth,GasGrowth,OilGrowth,AverageTemperature,AverageTemperatureUncertainty,TempMinus1,TempMinus2,TempMinus5,log_Gas,log_Coal,log_Oil,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty,Gas cumsum,Coal cumsum,Oil cumsum
29,2009,5.673677e+09,2.902559e+12,6.787239e+09,4.712376e+09,-2063.064,-558.672,-586.854,-917.537,15.336614,0.268833,15.286109,15.456948,15.150121,28.696614,22.638310,22.273458,9.505250,0.085917,15.266583,0.107750,3.796917,0.126000,15.827167,0.058917,5.770871e+13,1.254781e+11,1.108386e+11
30,2010,5.729765e+09,3.174016e+12,7.305800e+09,4.866603e+09,5350.160,1837.893,2110.871,1401.398,15.289953,0.265576,15.336614,15.286109,15.154312,28.786019,22.711934,22.305662,9.703083,0.083417,15.449000,0.103417,4.023917,0.115667,15.895500,0.058583,6.088272e+13,1.327839e+11,1.157052e+11
31,2011,5.785305e+09,3.264771e+12,7.729972e+09,4.904080e+09,3290.304,2019.350,768.775,502.187,15.255487,0.275816,15.289953,15.336614,15.326454,28.814211,22.768371,22.313333,9.516000,0.082000,15.284833,0.114333,3.827667,0.136583,15.769500,0.059000,6.414749e+13,1.405138e+11,1.206093e+11
32,2012,5.842192e+09,3.333059e+12,7.961497e+09,4.973526e+09,1591.055,190.067,835.661,565.328,15.247377,0.361260,15.255487,15.289953,15.456948,28.834912,22.797883,22.327395,9.507333,0.083417,15.332833,0.107333,3.756167,0.145333,15.802333,0.061500,6.748055e+13,1.484753e+11,1.255828e+11
33,2013,5.898015e+09,3.373866e+12,8.033213e+09,5.051871e+09,1876.422,828.760,530.413,517.259,15.906935,0.458943,15.247377,15.255487,15.286109,28.847080,22.806850,22.343024,9.606500,0.097667,15.373833,0.115500,3.911333,0.149833,15.854417,0.064667,7.085442e+13,1.565085e+11,1.306347e+11


In [58]:
def train_model(X,y,type="Ridge"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    if type == "LinearRegression":
        lm = LinearRegression().fit(X_train,y_train)
    elif type == "ElasticNetScaled":
        pipe = Pipeline([('scaler', StandardScaler()), ('EN', ElasticNet(alpha=0.1))])
        lm = pipe.fit(X_train, y_train)
    elif type == "ElasticNet":
        lm = ElasticNet(alpha=0.1).fit(X_train,y_train)
    elif type == "Ridge":
        lm = Ridge(alpha=0.1).fit(X_train,y_train)
    elif type == "Lasso":
        lm = Lasso(alpha=0.1).fit(X_train,y_train)
    elif type == "OMP":
        lm = OrthogonalMatchingPursuit().fit(X_train,y_train)
    #elif type == "GaussianProcess":   <- this didn't work, so maybe I am doing something wrong.
    #    kernel = DotProduct() + WhiteKernel()
    #    lm = GaussianProcessRegressor(kernel=kernel,random_state=0,alpha = 1e-5).fit(X, y)
    #    print("Gscore: ",lm.score(X, y))
    else:
        lm = LinearRegression().fit(X_train,y_train)
    print("Model Type: ",type)     
    print("r2 score: ",r2_score(lm.predict(X_test),y_test))
    print("mse: ",mean_squared_error(lm.predict(X_test),y_test))
    
    return lm, X_train, X_test, y_train, y_test
    

In [59]:
pca_data_columns = ['Population',
 'Gas consumption',
 'Coal consumption',
 'Oil consumption',
 'Gas cumsum',
 'Coal cumsum',
 'Oil cumsum',
 'FossilFuelGrowth',
 'CoalGrowth',
 'GasGrowth',
 'OilGrowth',
 'TempMinus1',
 'TempMinus2']
 #'TempMinus5',


pca_df = df[pca_data_columns+['AverageTemperature']].dropna()

target_columns = ['AverageTemperature']

# create a base classifier used to evaluate a subset of attributes

# create the RFE model and select 3 attributes
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=10)
rfe = rfe.fit(pca_df[pca_data_columns], pca_df[target_columns])
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[ True  True  True  True  True  True  True False  True False False  True
  True]
[1 1 1 1 1 1 1 3 1 2 4 1 1]


In [60]:
data_columns = list(rfe.feature_names_in_[rfe.support_])
target_columns = ['AverageTemperature']

model_df = df[data_columns+['AverageTemperature']].dropna()

y = model_df[target_columns]
X = model_df[data_columns]
for model in ['LinearRegression','Lasso','ElasticNet','OMP','Ridge']:
    lm, X_train, X_test, y_train, y_test = train_model(X,y,model)

Model Type:  LinearRegression
r2 score:  0.8855876951129755
mse:  0.03642365402014431
Model Type:  Lasso
r2 score:  0.8280756568025082
mse:  0.0398913042263386
Model Type:  ElasticNet
r2 score:  0.828065370162928
mse:  0.039891928030044134
Model Type:  OMP
r2 score:  0.7866251039994681
mse:  0.06820216709280362
Model Type:  Ridge
r2 score:  0.887736376068946
mse:  0.03538607684919299


In [61]:
score_df = df.loc[X_test.index][['AverageTemperature']]
score_df['Model Prediction'] = lm.predict(X_test)
score_df['Minus1 Prediction'] = df.loc[X_test.index]['TempMinus1']
score_df = score_df.sort_values(by = 'AverageTemperature')
score_df[['AverageTemperature','Model Prediction','Minus1 Prediction']]

,AverageTemperature,Model Prediction,Minus1 Prediction
17,14.900671,14.939040,14.479931
14,15.049933,14.656628,14.510896
32,15.247377,15.491680,15.255487
31,15.255487,15.326292,15.289953
19,15.267058,15.346750,15.256991
26,15.326454,15.104865,15.154312
27,15.456948,15.195905,15.326454
6,15.936502,16.036697,15.837643
2,15.986446,16.079545,16.058908
11,16.016611,16.175804,16.562689


In [62]:
print("Model Scores: ")
print("r2 score: ",r2_score(lm.predict(X_test),y_test))
print("mse: ",mean_squared_error(lm.predict(X_test),y_test))
print()
print("Simple Lag Model Scores: ")
print("r2 score: ",r2_score(X_test['TempMinus1'],y_test))
print("mse: ",mean_squared_error(X_test['TempMinus1'],y_test))

Model Scores: 
r2 score:  0.887736376068946
mse:  0.03538607684919299

Simple Lag Model Scores: 
r2 score:  0.8040510061971596
mse:  0.08081815163263606
